In [1]:
import pandas as pd

from sklearn.preprocessing import StandardScaler, OneHotEncoder, FunctionTransformer
from sklearn.compose import ColumnTransformer, make_column_transformer
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.model_selection import RandomizedSearchCV

from xgboost import XGBClassifier

from joblib import dump, load

RANDOM_STATE = 7
CV = 3

pd.set_option('display.max_columns', None)


In [2]:
df = pd.read_csv('data/train.csv',index_col='id')
test = pd.read_csv('data/test.csv',index_col='id')


# PROCESS DATA

In [5]:
from custom_transformers import set_buckets,BMITransformer,APTransformer
# вся предобработка вынесена в отдельный файл


In [6]:
X = df.drop('cardio',axis=1)
y = df.cardio


In [4]:
# добавим два тестовых признака
def test_feature(X_):
    X_ = X_.copy()
    X_['test_feature']   = X_['ap_hi'] * X_['ap_lo']
    X_['test_feature_2'] = X_['gluc']  * X_['cholesterol']
    
    return X_

In [7]:
# предобработка данных – коррекция выбросов, заполнение пропущенных значений
clean_the_data = make_pipeline(
    FunctionTransformer(set_buckets),
    BMITransformer(),
    APTransformer(),
    FunctionTransformer(test_feature),
)

# кодирование и масштабирование признаков
to_scale  = ['age',
             'bmi',
             'weight_bucket',
             'ap_hi',
             'ap_lo',
             'test_feature',
             'test_feature_2',
            ]

to_encode = ['gender']

to_pass = ['smoke',
           'alco',
           'active',
           'cholesterol',
           'gluc',
          ]

transformers = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(drop='first'), to_encode),
    ('num', StandardScaler(), to_scale),
    ('pss', 'passthrough', to_pass)
])

# общий пайплайн по предобработке
preprocess = make_pipeline(
    clean_the_data,
    transformers
)    


# XGBOOST

In [3]:
# вывод важности признаков
def show_feature_importances(model):
    all_names = model[0][1].get_feature_names_out()
    display(pd.DataFrame({'features':all_names,
                          'feature_importances':model._final_estimator.feature_importances_}) \
              .sort_values(by='feature_importances',ascending=False))


In [8]:
%%time

# пайплайн модели
pipe_xgb = Pipeline(steps=[
    ('pre', preprocess),
    ('xgb', XGBClassifier(random_state=RANDOM_STATE))
])    

# гиперпараметры модели для оптимизации
xgb_params = {
    'xgb__learning_rate': [0.01, 0.05, 0.1, 0.25, 0.5],
    'xgb__max_depth': [3, 5, 8, 10, 12, 16],
    'xgb__n_estimators': range(200, 601, 20),
    'xgb__gamma': [0, 0.1, 0.5],
    'xgb__subsample': [0.2, 0.4, 0.6, 0.8, 1],
    'xgb__colsample_bytree': [0.2, 0.4, 0.6, 0.8, 1],
    'xgb__reg_alpha': [0, 1, 2, 3, 4, 5, 10],
    'xgb__reg_lambda': [0, 1, 2, 3, 4, 5, 10]
}

# поиск оптимальной модели по roc_auc
xgb = RandomizedSearchCV(pipe_xgb, 
                 xgb_params, 
                 n_iter=50,              
                 scoring='roc_auc', 
                 verbose=1,
                 n_jobs=-1, 
                 cv=CV)

xgb.fit(X,y)

# вывод лучшей метрики и параметров
display(xgb.best_score_)
display(xgb.best_params_)

xgb_best = xgb.best_estimator_

# вывод важности признаков
show_feature_importances(xgb_best)


Fitting 3 folds for each of 50 candidates, totalling 150 fits


0.803849196090708

{'xgb__subsample': 0.6,
 'xgb__reg_lambda': 1,
 'xgb__reg_alpha': 3,
 'xgb__n_estimators': 600,
 'xgb__max_depth': 3,
 'xgb__learning_rate': 0.05,
 'xgb__gamma': 0,
 'xgb__colsample_bytree': 0.6}

,features,feature_importances
4,num__ap_hi,0.394997
6,num__test_feature,0.206722
11,pss__cholesterol,0.138097
1,num__age,0.050053
7,num__test_feature_2,0.045975
5,num__ap_lo,0.032972
10,pss__active,0.027387
3,num__weight_bucket,0.022217
2,num__bmi,0.018752
12,pss__gluc,0.018129


CPU times: user 42.9 s, sys: 10.4 s, total: 53.3 s
Wall time: 13min 17s


# RESULTS

In [11]:
# экспорт результатов – модели, предобработки и предсказания

def export_result(model):
    model.fit(X,y)
    predict_proba = model.predict_proba(test)[:,1]

    pd.DataFrame(index=test.index,
                 data=predict_proba,
                 columns=['cardio']) \
      .to_csv('test_predict_proba.csv')
        
    '''
    Коррекция признаков, поиск ошибок, выбросы, заполнение пропусков 
    – эти процедуры актуальны для датасетов, например для архивных данных.
    
    Для интерактивного приложения с предсказанием 
    по единственному набору – можно ограничить предобработку 
    стандартным масштабированием и кодированием.
    
    Поэтому выгружаем не целиком весь процесс, а только избранное )
    '''

    dump(model[0][1], 'scaler_encoder.joblib') 
    dump(model[1], 'XGB_classifier.joblib') 


In [12]:
export_result(xgb_best)
